<h1>PEST: pyLIQTR's Electronic Structure Tools </h1>

<font size="3">

<hr>
DISTRIBUTION STATEMENT A. Approved for public release. Distribution is unlimited.

This material is based upon work supported by the Under Secretary of Defense for
Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions,
findings, conclusions or recommendations expressed in this material are those of the
author(s) and do not necessarily reflect the views of the Under Secretary of Defense
for Research and Engineering.

© 2023 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part
252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government
rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed
above. Use of this work other than as specifically authorized by the U.S. Government
may violate any copyrights that exist in this work.
<hr><hr>
</font>

<b>PEST</b> is a Julia-based library of  electronic structure codes for `pyLIQTR`.  These can be used as standalone Julia-based scripts or run on the command line.  This notebook demonstrates general use of `PEST`, while Python interfaces are described in the `pest-python.ipynb` notebook.  

<h3><b><u>1. Overview</u></b></h3>

While `PEST` will ultimately develop into a multi-functional tool, the current emphasis is on the preparation of electronic structure Hamiltonians for quantum algorithms.  We outline the general nature and scope of this approach in the following.

<b><u>What can PEST do for resource analysis?</u></b>

`PEST` can generate coefficients for (periodic) electronic structure Hamiltonians in either a nonlocal <b>  plane-wave (PW) basis </b> or in a semi-local <b> dual plane-wave (DPW) basis</b>.  The required inputs are (1) a set of atomic coordinates for a given material or molecule; (2) a description of the supercell geometry for a periodic system; and (3) cutoffs that specify the number of basis functions that are to be constructed using a $k$-point mesh or real-space grid:

<center>
<img src="./figures/fig-pestflow.png" width=800>
</center>



<b><u>What should I know about these frameworks?</u></b>

There are three common basis schemes for electronic structure calculations.  We  can be broadly classify these according to the real-space locality of their basis elements.  Local basis sets are generally defined using Gaussian or exponentially decaying functions (termed Gaussian and Slater-type orbitals, respectively)  and introduced as isolated atomic orbitals. This  arrangement is particularly natural for aperiodic systems or single molecules.  Following this logic, nonlocal  bases are often efficient for periodic or spatially-extended systems.  <b>Plane waves (PW)</b> are a popular choice for this case, and they may be supplemented with <i>pseudopotentials</i> that average out unimportant features and reduce the required basis size.  This partition is not strict $-$ isolated molecules can be described using plane waves provided that we introduce interstitial "vacuum padding" between periodic images.  However, there may be  tradeoffs in overhead due to an  inefficient description of the underlying physics.

<center>
<img src="./figures/fig-wavefunctions.png" width=800>
</center>

Other arrangements can bridge this partition.  Mixed bases such as Gaussian plane-waves (GPWs) have intermediate locality and exhibit robust performance for many systems.  Another intermediate strategy is derived from the Fourier transform of a plane-wave scheme, which  gives a corresponding <b>dual plane-wave (DPW)</b> basis.  These have intermediate locality in both real-space and momentum-space (conversely, plane-waves are nonlocal in real-space and local in momentum-space).  Notably, the underlying DPW functions are centered on the vertices of a regular, real-space grid as opposed to being fully spatially extended.  This arrangement can deliver high efficiency for certain electronic structure problems, such as those involving surfaces or bulk materials.  DPWs also provide  an efficient factorization of second-quantized Hamiltonians, ultimately enabling fault-tolerant ground-state estimation algorithms with linear $T$-complexity in problem scale. While the size of a PW basis is fixed by a cutoff $E_\text{cut}$ for the highest-energy plane wave states, the size of a DPW basis is determined by the spacing $a_0$ between real-space grid points.  There is a direct correspondence between these measures, which we discuss below, that allows us to extrapolate classical PW benchmarks to DPWs. A more comprehensive description of these methods is found in [1,2].  

The `PEST` code supports both <b>plane-wave</b> bases and their <b>dual plane-wave</b> counterparts.

<b><u>How do I describe my material / molecule?</u></b>

The PEST code is well-suited to periodic systems.  However, these methods can also be used for isolated molecules with minor modifications. The required inputs are uniformly (i) a set of coordinates that describe the atoms in a "supercell" and (ii) a set of vectors that specify the geometry of this cell.  In this context, a supercell is simply the minimal repeating unit that is explicitly described by our calculations.  This can range from the unit cell of a bulk material to a large slab  that supports surface reactions.  The cell vectors define the extent of this periodic unit along a fixed set of orthorhombic axes (other cells are a work in progress).


  The size of a supercell is determined by the physical problem as well as the desired level of computational detail.  For instance, this cell must accommodate all chemical reactants, key material features, and  any encapsulating solvent (if needed).  There is also an interplay between the cell contents and the basis scheme, particularly with regard to quantitative accuracy. For instance, a minimal supercell $-$ containing only one unit cell of some material $-$ would mandate a fine $k$-point mesh when using plane-wave basis.  This dense mesh is required to properly sample states throughout the Brillouin zone.  The analogous dual plane-wave calculations require a supercell containing many repeated unit cells in order to accommodate extended electronic states and avoid self-interaction artifacts between periodic images.



<b><u>How do I choose an energy cutoff / grid spacing?</u></b>

By referring to the classical electronic structure literature!  

Most chemical systems (or a close analogue) have been benchmarked using plane-wave density functional theory (DFT).  A key quantity is the converged PW energy cutoff $E_\text{cut}$, which ultimately determines the size of our basis set.  However, this is only part of the story $-$ classical methods generally use pseudopotentials to eliminate unimportant yet computationally expensive features.  These reduce the required energy cutoffs to the order of 10-100 Ry instead of a prohibitive  $10^5-10^6$ Ry that would  otherwise be required.  
 
 Our implementation follows the "pure" PW/DPW approach from the quantum algorithms in  [1,2] and does not (yet) account for pseudopotentials.  Using  pseudpotential-derived cutoffs with our code will give inaccurate values for some Hamiltonian coefficients.  However, `PEST` will  generate the correct operator terms and hence facilitate robust resource estimates.  This is not unduly speculative, since the formal correspondence PWs and DPWs will, in principle, allow us to construct a pseudopotential-based quantum algorithm.

Low $T$-complexity quantum algorithms require a dual plane-wave basis in lieu of the plane-wave arrangement discussed above.  Fortunately, the duality between PWs and DPWs also allows us to specify an equivalent dual grid spacing:

\begin{equation}
a_0 = \gamma \sqrt{2 \pi^2 / E_\text{cut}} \sim \gamma \lambda_\text{cut}.
\end{equation}

 Here, $\lambda_\text{cut}$ is the wavelength corresponding to the highest-energy PW mode.  The factor $\gamma$ specifies if we sample at this wavelength ($\gamma = 1.0$) or half of this wavelength ($\gamma = 0.5$).  An ideal value would depend on pseudopotential details, though likely this would be closer to $\gamma = 0.5$.  Thus, this expression gives us a robust means to specify DPW grids in terms of a classical reference with prescribed accuracy.


<h3><b><u>2. Jupyter Notebook Examples: DPWs</u></b></h3>

The following examples are based on the <b>dual plane-wave</b> approach, since this enables highly-efficient quantum algorithms for second-quantized systems.

We begin by loading the `PEST` module.  The path to the `PEST` source must be specified if it hasn't been installed using Julia's package manager (using  `pest_path=` and `Pkg.develop()` or `LOAD_PATH` as in the commented lines):

In [2]:
pest_path="../../PEST/src"

push!(LOAD_PATH,pest_path)

# using Pkg
# Pkg.develop(pest_path)

using  PEST


<h4><i><b>Example I: Reactivity on a Mg (0001) Surface</b></i></h4>

Our first example addresses the reactivity of an Mg (0001) surface, with an eye toward corrosive processes.  Calculations are based on orthorhombic supercells containing a 4x4x2 or 8x8x2 repeat of the Mg unit cell (green).  The lattice vectors are chosen to define an extended slab in the $xy$-plane while leaving a vacuum gap normal to the surface. This gives a periodic arrangement reflected at right in the figure below:


<center>
<img src="./figures/figure-slab.png" width=800>
</center>

The extension to a fully periodic material (e.g., bulk solid) would be straightforward - we simply omit the vacuum gap!

We will now build a DPW Hamiltonian for a 4x4x2 Mg (0001) slab using `PEST`.  This example uses an unphysically low cutoff of $E_\text{cut} = 5.0\,\text{Ry}$ for fast execution (the multithreaded, command-line version of `PEST` is preferred for production calculations).  A more reasonable range would be $E_\text{cut} = 30.0\,\text{Ry}$ to $60.0\,\text{Ry}$ when shooting for consistency with Vanderbilt ultrasoft pseudopotentials.   A scaling factor of $\gamma = 1.0$ is used, which gives a coarser DPW grid.



A simple approach uses `calc_hamiltonian` to generate coefficients for a DPW Hamiltonian.  There are three required arguments:

1. The path to an `.xyz` file containing a set of atomic coordinates (in Angstroms).
2. Either: (1) a target spacing between points in the real-space DPW basis grid $a_0$ (in Bohr; `a0=` keyword) or (2) an equivalent plane-wave cutoff $E_\text{cut}$ (in Rydberg; `E_cut=` keyword).  The grid spacing  $a_0 = \gamma \sqrt{2 \pi^2 / E_\text{cut}} $ is automatically calculated when specifying the latter.
3. A vector containing the vacuum padding between periodic images (`pad=` keyword) or a direct specification of the supercell lengths  (`lengths=`).  These values are specified in Angstroms.

There are also optional inputs:

1. <b>Usually Required:</b> The scaling factor $\gamma$ when converting from a cutoff to grid spacing [Default: `scale=1.0`].
2. A numerical threshold `eps_z` that determines when numerical output should be rounded to zero [Default: `eps_z=1e-10`]

Our methods will compute coefficients for Fermionic and Pauli string representations by default (a description is given at the end of this notebook).  Putting these points into practice:

In [10]:
ham,grid = calc_hamiltonian( "../../data/geometries/magnesium-4x4x2-orthorhombic.xyz",
                             lengths=[12.83708, 11.11724, 15.631000],
                             E_cut = 5,
                             scale=1.0   );

###  Requested a DPW Hamiltonian.

	Energy Cutoff  =  5 Ry 
	Grid Spacing   =  2.809926 Bohr   (scaling = 1.0)
	Grid           =  9 x 8 x 11  ( 792 basis functions ).

###  Loaded geometry for 64 atoms.

	Lattice Dimensons  =  24.261 x 21.011 x 29.542 (Bohr)
	Actual Spacing     =  2.696 x 2.626 x 2.686 (Bohr)

###  Building grid and mesh.

	Building grid distance permutations: 
	Applying symmetry operations: 


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


	Generated a minimal set of 4163 displacement vectors.
	Generated a minimal set of 152 distances.

###  Calculating the Hamiltonian.

	Calculating Kinetic & E-E Interaction Terms:


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


	Calculating E-Ion Terms:


Progress: 100%|█████████████████████████████████████████| Time: 0:00:30


The calculation takes roughly one minute.  This returns a `FermionicHamiltonian` container which holds a minimal representation of the Hamiltonian coefficients (for both Fermionic operator terms and Pauli strings) and a `GridData` container that holds the basis grid.

The fermionic pair coefficients $T_{p,q} = T(r_p-r_q)$ can be returned for arbitrary indices $p,q \in \mathbb{N}$ using the function `pair_coeff_lookup()`.  For instance to extract $T_{1,3}$  (or, analogously $V_{3,2}$) we would write:

In [4]:
T_1_3 = pair_coeff_lookup(1,3,grid,ham.T)
V_3_2 = pair_coeff_lookup(3,2,grid,ham.V)

println("T_1_3 = ", T_1_3)
println("V_3_2 = ", V_3_2)

T_1_3 = 0.03254909279736752
V_3_2 = 0.14011870407728652


The Hamiltonian and grid can be written to an HDF5 file using a pair of helper functions:

In [7]:
write_hamiltonian_hdf5("ham.hdf5",ham);
write_grid_hdf5("grid.hdf5",grid);

Similar routines are provided to read these objects:

In [8]:
ham  = read_hamiltonian_hdf5("ham.hdf5");
grid = read_grid_hdf5("grid.hdf5");

This data can also be manipulated in Python scripts, with coefficient lookup and HDF5 management provided by the methods in `filename.py` (see  `filename_explain.ipynb` for an exposition).  We also provide notebooks that show how to use these data in a resource estimation workflow.

<h4><i><b>Example II: Cyclic Ozone  in a  Fullerene Cage  </i></b></h4>

Isolated molecules can also be handled in a PW or DPW framework, though they require a bit of additional care.  In this case, we must decouple the molecule from its periodic images by using a suitably large vacuum gap.  This should be sufficient to attenuate both electrostatic coupling as well as wavefunction 'leakage' across periodic boundaries:

<center>
<img src="./figures/figure-o3-c60.png" width=800>
</center>

We will use `PEST` to calculate the DPW Hamiltonian for an isolated $\text{C}_{60}$-$\text{O}_3$ system.  The `calc_hamiltonian` function sets the periodic cell dimensions equal to the spatial extent of the molecule if no user override is given.  We can add a vacuum layer to this arrangement by using the `pad=` keyword, which accepts a array containing the additional elongation for each lattice vector (in Angstroms).  For instance, to perform a calculation with $E_\text{cut} = 5.0\,\text{Ry}$, $\gamma = 1.0$ and a $5.0 \,\text{\AA}$ padding along each direction:

In [9]:
ham,grid = calc_hamiltonian( "../../data/geometries/C60-o3-opt.xyz",
                             pad=[5.0,5.0,5.0],
                             E_cut = 5,
                             scale=1.0   );

###  Requested a DPW Hamiltonian.

	Energy Cutoff  =  5 Ry 
	Grid Spacing   =  2.809926 Bohr   (scaling = 1.0)
	Grid           =  9 x 9 x 9  ( 729 basis functions ).

###  Loaded geometry for 63 atoms.

	Lattice Dimensons  =  22.516 x 23.137 x 23.194 (Bohr)
	Actual Spacing     =  2.502 x 2.571 x 2.577 (Bohr)

###  Building grid and mesh.

	Building grid distance permutations: 
	Applying symmetry operations: 


	Generated a minimal set of 2457 displacement vectors.
	Generated a minimal set of 116 distances.

###  Calculating the Hamiltonian.

	Calculating Kinetic & E-E Interaction Terms:


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


	Calculating E-Ion Terms:


Progress: 100%|█████████████████████████████████████████| Time: 0:00:22


A more efficient treatment of isolated molecules would also utilize Coulomb cutoffs, which remain in the pipeline (<b>TODO</b>). 

<h3><b><u>3. Command Line Usage (Multithreaded)</u></b></h3>

A more computationally efficient approach is to run `PEST` external to a Jupyter notebook (i.e., on the command line).  This allows us to exploit Julia's native multithreading and gain a marked speedup when calculating the electron-ion potential.  In principle, this can also be done using Jupyter notebook.  However, this is less reliable in practice since it is highly dependent on the Jupyter interpreter and environment.

Command-line execution is handled by a Bash script called `pest_commander.`  This serves as a wrapper for the Julia JIT compiler and handles thread specification.  This is less full-featured than direct use of the `PEST` modules, though the capabilities are  sufficient for most problems.



<b>NOTE:</b> The `pest_commander` shell wrapper can be found in the `bin` directory.  Additionally, the environment variable `PEST_DIR` must point to the location of the PEST source (or the location of `pest_commander.jl`) if PEST has been installed as a Julia package.  This can be set using "`export PEST_DIR=...`" on the command line for `sh`/`bash/zsh` or using "`setenv PEST_DIR ...`" for `csh`/`tcsh` (double quotes omitted and ellipses replaced with the path).

The command-line arguments to `pest_commander` follow from the discussion above. It is important to note that every argument is specified via a flag:

<font size="3">

        > ./pest_commander --help

        usage: pest_commander.jl [--coords COORDS] [--e_cut E_CUT] [--a0 A0]
                                 [--scale SCALE] [--pad PAD PAD PAD]
                                 [--lengths LENGTHS LENGTHS LENGTHS]
                                 [--eps_z EPS_Z] [--output_pfx OUTPUT_PFX] [-h]

        optional arguments:
         --coords COORDS       Path to coordinate file in XYZ format (units
                               in Angstroms).  Cell vectors calculated from
                               minimal bounding box in cartesian frame,
                               unless --lenghts are specified. (default: "")
         --e_cut E_CUT         Plane-wave cutoff energy E_cut [Rydberg]
                               (type: Float64, default: 30.0)
        --a0 A0                Real-space grid spacing a0 [Bohr] (type: Float64)
        --scale SCALE          Rescaling factor when real-space lattice
                               constant is derived from E_cut [ a0' = scale * a0 ] 
                               (type: Float64, default: 1.0)
        --pad PAD PAD PAD      Vacuum padding added to supercell lengths (if
                               derived from input coordinates)  [Angstrom]
                               (type: Float64)
        --lengths LENGTHS LENGTHS LENGTHS
                               Lengths defining supercell along each
                               cartesian coordinate [Angstrom] (type:
                               Float64)
        --eps_z EPS_Z          Specify cutoff for rounding small coefficients
                               to zero (default = 1e-10) (type: Float64,
                               default: 1.0e-10)
        --output_pfx OUTPUT_PFX
                               Prefix for grid and hamiltonian output files.
        -h, --help             show this help message and exit
</font>

 Note that we now have an argument `--output_pfx` which specifies a prefix string that is appended to HDF5 datafiles (`<prefix>.ham.hdf5, <prefix>.grid.hdf5`).  These outputs are always generated during command-line usage.
 
 The maximum number of runtime threads can be changed by editing `pest_commander` and changing the environment variable `NUM_THREADS=8`.    

To run our cyclic ozone example the command line we would use (now with a $E_\text{cut} = 30\, \text{Ry}$ cutoff):

<font size="3">

        >  ./pest_commander --coords ../../data/geometries/c60-o3-opt.xyz --e_cut 30.0 --scale 1.0 --pad 5.0 5.0 5.0 --output_pfx c60-o3-opt-dpw

</font>

This generates a descriptive set of outputs and progress indicators:

<font size="3">


        ./pest_commander --coords ../../data/geometries/c60-o3-opt.xyz --e_cut 30.0 --scale 1.0 --pad 5.0 5.0 5.0 --output_pfx c60-o3-opt-dpw

        PEST COMMANDER    ( Threads = 8 )

        ###  Requested a DPW Hamiltonian.

            Cutoff Equiv.  =  30.0 Ry
            Grid Spacing   =  1.147147 Bohr   (scaling = 1.0)
            Grid           =  20 x 21 x 21  ( 8820 basis functions ).

        ###  Loaded geometry for 63 atoms.

            Lattice Dimensons  =  22.516 x 23.137 x 23.194 (Bohr)
            Actual Spacing     =  1.126 x 1.102 x 1.104 (Bohr)

        ###  Building grid and mesh.

            Building grid distance permutations:
            Applying symmetry operations:
        Progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Time: 0:00:17
            Generated a minimal set of 34457 displacement vectors.
            Generated a minimal set of 726 distances.

        ###  Calculating the Hamiltonian.

            Calculating Kinetic & E-E Interaction Terms:
        Progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Time: 0:00:05
            Calculating E-Ion Terms:
        Progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| Time: 0:13:11

        ###  Writing hamiltonian and grid in HDF5 format.

        DONE!!

</font>


<h3><b><u>4. Technical Details</u></b></h3>

The following describes the physical scope of our methods and the coefficients that are output by `PEST`.

It is important to qualify our approximations from the outset.  First and foremost we restrict to the adiabatic limit — that is, where electron dynamics substantially outpace those of the heavier nuclei (aka "ions"). This is accomplished by invoking the Born-Oppenheimer approximation, which is suitable for a broad range of static and quasi-static problems. A more comprehensive approach might go beyond this restriction and treat non-adiabatic dynamics. That is not attempted here.

A second limitation is our definitions originate from a <b>nonlocal, all-electron plane wave (PW) basis</b> expansion. This level of detail is both unnecessary and intractable for most calculations of matter under non-extreme conditions.  The  problem is that the sharp potential near atomic nuclei will induce rapid wavefunction oscillations in their vicinity (the so-called "nuclear cusp").  These, in turn, require the inclusion of very high-frequency basis modes and thus a high energy cutoff (or a very fine real-space grid).  Classical electronic electronic codes generally avoid this issue by using pseudopotentials to smooth the nuclear potential and mitigate these oscillations.  The end result is that we can use a much lower energy cutoff and thus a smaller basis.

While a DPW representation has not been formalized for pseudopotential methods, it will be related to its plane-wave counterpart by a Fourier transform (in practice there are some other subtleties, e.g., regarding norm-conservation, etc.).  We will ignore these complications to first order, and assume that a given plane-wave pseudopotential cutoff $E_\text{cut}$ will map directly to a real-space grid spacing $a_0$. Other basis expansions could also be used (wavelets, inhomogeneous grids), though the tradeoffs inherent to each remain an open question.
    
Given these conventions, the $N$-orbital / basis element  <b>plane-wave electronic structure Hamiltonian</b> is:

\begin{equation}
H = \underbrace{\frac{1}{2} \sum_{p,\sigma} k_p^2 \, c^\dagger_{p,\sigma} c_{p,\sigma}}_{\text{Kinetic}} - \underbrace{\frac{4\pi}{\Omega} \sum_{\substack{p\neq q,j\sigma }}  \Big(\zeta_j \, \frac{e^{\imath k_{p-q} \cdot R_j}}{k_{p-q}^2} \Big)\, c^\dagger_{p,\sigma} c_{q,\sigma}}_{\text{Electron-Ion}} +  \underbrace{\frac{2\pi}{\Omega} \sum_{\substack{(p,\sigma) \neq (q,\sigma') \\ \nu \neq 0}} \Big(\frac{1}{k_\nu^2} \Big)\,\, c^\dagger_{p,\sigma} c^\dagger_{q,\sigma} c_{q+\nu,\sigma'} c_{p-\nu,\sigma}}_{\text{Electron-electron}}
\end{equation}

where $c_{p,\sigma}^\dagger, c_{p,\sigma}$ are the fermionic creation and annihilation operators for an electron in the plane wave state $\psi_p (r) = \Omega^{-1/2}  e^{\imath k_p \cdot r}$ with associated position $r$ and wavevector $k_\nu = 2\pi \nu / \Omega^{1/3}$.  The vector index $p$  is then required to take values in $[ -N^{1/3}, N^{1/3}]$ while the spin  component is indexed by $\sigma \in \{\uparrow,\downarrow\}$.  The first term is then the "kinetic" contribution in this expansion, while the second term captures the electron-ion $^1$ interaction for a set of atoms with atomic number $\zeta_j$ located at spatial coordinates $R_j$.  Finally, the last term corresponds to the electron-electron interaction.  The first term scales as $O(N)$, the second as $O(N^2)$, and the last as $O(N^3)$ (though this would be $O(N^4)$ in an atomic orbital basis).

We can map this Hamiltonian to real-space by performing a discrete Fourier transform of its basis functions.  This redefines the electronic structure problem in terms of a semi-local basis that lives on a regular, real-space grid at points  $r_p$.  Since these new functions are dual (by Fourier transform) to plane waves, they have been colloquially termed a <b>dual plane-wave (DPW)</b> basis. The DPW Hamiltonian superficially resembles its nonlocal counterpart:

\begin{align}
H &=  \sum_{p,q,\sigma}T(p-q)\, c^\dagger_{p,\sigma} c_{q,\sigma} -\sum_{p,\sigma }  U(p) \, n_{p,\sigma} + \sum_{(p,\sigma) \neq (q,\sigma') } V(p-q)\, n_{p,\sigma} n_{q,\sigma'} \\
&=\underbrace{\frac{1}{2N} \sum_{\nu,p,q,\sigma} k_\nu^2 \, \cos \big[ k_\nu \cdot r_{p-q} \big] c^\dagger_{p,\sigma} c_{q,\sigma}}_{\text{Kinetic}} - \underbrace{\frac{4\pi}{\Omega} \sum_{\substack{p,\sigma \\ j,\nu \neq 0 }}  \frac{\zeta_j \, \cos \big[ k_\nu \cdot (R_j - r_p)\big]}{k_\nu^2} \, n_{p,\sigma}}_{\text{Electron-Ion}} + \underbrace{\frac{2\pi}{\Omega} \sum_{\substack{(p,\sigma) \neq (q,\sigma') \\ \nu \neq 0}} \frac{\cos \big[k_\nu \cdot r_{p-q} \big]}{k_\nu^2} n_{p,\sigma} n_{q,\sigma'}}_{\text{Electron-electron}}
\end{align}

where the first line introduces our notation for the kinetic $T(p-q)$, electron-electron $V(p-q)$, and electron-ion $U(p)$ coefficients.  Note that the fermionic operators have been reordered in the electron-electron term, which gives an efficient factorization for translationally-invariant (periodic) systems.  Taken collectively, these accommodations reduce the electron-electron scaling to $O(N^2)$ while increasing the kinetic scaling to  $O(N^2)$.  This tradeoff is a good compromise since the interaction terms have greater overhead for all but the smallest of systems.  

We must map Fermions to qubits in order to implement this on a quantum computer.  This can be accomplished using various strategies, of which the simplest is probably the Jordan-Wigner transform (JWT).  While the choice of a given transform will have a nontrivial impact on product-formula based simulation algorithms, the leading overhead for qubitized methods is fixed by the number of terms in the Hamiltonian.  To see this in practice, we  use the JWT to rewrite our Hamiltonian in terms of Pauli string operators:

\begin{align}
H &=  \sum_{\substack{p,\sigma \\ \nu \neq 0 }} \Big(  \frac{\pi}{\Omega \, k_\nu^2} - \frac{k_\nu^2}{4N} + \frac{2\pi}{\Omega} \sum_j \frac{\zeta_j \, \cos \big[ k_\nu \cdot (R_j - r_p)\big]}{k_\nu^2} \Big) \cdot Z_{p,\sigma}  \\ & \,\,\, + \frac{\pi}{2\Omega } \sum_{\substack{(p,\sigma) \neq (q,\sigma') \\ \nu \neq 0}} \Big( \frac{\cos \big[ k_\nu \cdot r_{p-q} \big] }{k_\nu^2} \Big) \cdot Z_{p,\sigma} \otimes Z_{p,\sigma'} + \sum_{\nu \neq 0} \Big( \frac{k_\nu^2}{2} - \frac{N \pi} {\Omega k_\nu^2}) \cdot I \\
& + \frac{1}{4N} \sum_{\substack{p\neq q \\ \nu,\sigma}} \Big( k_\nu^2 \cos \big[k_\nu \cdot r_{p-q}\big] \Big) \cdot (X_{p,\sigma} \otimes \vec{Z}_{p,q,\sigma} \otimes X_{q,\sigma} + Y_{p,\sigma} \otimes \vec{Z}_{p,q,\sigma} \otimes Y_{q,\sigma} )
\end{align}

Here $(X/Y/Z)_{p,\sigma}$ is a Pauli operator acting on the qubit designated for the grid point $p$ with spin index $\sigma$. We use the shorthand $\vec{Z}_{p,q,\sigma} = Z_{p+1,\sigma} \otimes \dots \otimes Z_{q-1,\sigma}$ to denote the Pauli Z-string spanning between qubits $p$ and $q$.  The simplest qubitization schemes use a Select-Prepare oracle strategy to encodes this expression on a term-by-term basis.  The use of this explicit LCU ends up being quite inefficient in practice.   To make this more efficient,  Select and Prepare queries can be modified to implicitly prepare Pauli strings by consuming the fermionic operator coefficients  $T(p-q)$, $V(p-q)$ and $U(p)$ as input.

Our approach is to calculate the DPW Hamiltonian using a fermionic operators from the outset.  These terms are then converted to Pauli string coefficients if requested.

<h5><b>References</b></h5>

<ol>
<li> R. Babbush, N. Wiebe, J. McClean, J. McClain, H. Neven, G. K.-L. Chen.  "Low-Depth Quantum Simulation of Materials." Phys. Rev. X <b>8</b>, 011044 (2018)</li>
<li> R. Babbush, C. Gidney, D. W. Berry, N. Wiebe, J. McClean, A. Paler, A. Fowler, H. Neven. "Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity." Phys. Rev. X <b>8</b>, 041015 (2018) </li>
</ol>

<h5><b>Footnotes</b></h5>

<ol>
<li> We will refer to the atomic nuclei as ions since they are treated as positive point charges equal to their atomic number.  This is a standard convention in the electronic structure literature.</li>
</ol>